In [1]:
from utils.schedule_utils import load_data_split,table_to_str,table_to_str_sql,find_intersection_and_add_row_id,Prepare_Data_for_Operator_Sequence,format_document,batch_rerank_scores,ROLLBACK,merge_clean_and_format_df_dict,retrieve_rows_by_subtables,process_error_analysis_list
import pickle
import numpy as np
import pandas as pd
import os
import json
from FlagEmbedding import FlagReranker
from tqdm import tqdm
from typing import Dict, Any
from utils.evaluator import Evaluator
from utils.prompt_generate import build_wikitq_prompt_from_df,evaluate_predictions,filter_dataframe_from_responses,fix_sql_query,match_subtables,retrieve_rows_by_subtables,build_tab_fact_prompt_from_df
from utils.async_llm import infer_prompts

/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [5]:
### 读取必要文件（step 2）
dataset_name = 'wikitq'
split = 'test'
tmp_save_path = 'datasets/schedule_test/wikitq'
dataset = load_data_split(dataset_name,split)
wikitq_df_processed = np.load(f'{tmp_save_path}/wikitq_df_processed.npy',allow_pickle=True).item()
samples_short, samples_medium, samples_long = find_samples_by_length_from_dict(wikitq_df_processed,tokenizer_path='../../../models/Qwen3-4B-Instruct-2507/')

In [6]:
wikitq_df = pd.read_csv('wikitq_test_result_4B.csv',index_col=0)
for index_list in [samples_short, samples_medium, samples_long]:
    acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df.iloc[index_list], dataset)
# 

100%|██████████| 2677/2677 [00:47<00:00, 55.82it/s]


Correct Samples: 2115; Total Samples: 2677
Accuracy: 79.01


100%|██████████| 1047/1047 [00:18<00:00, 56.60it/s]


Correct Samples: 783; Total Samples: 1047
Accuracy: 74.79


100%|██████████| 620/620 [00:12<00:00, 50.08it/s]

Correct Samples: 429; Total Samples: 620
Accuracy: 69.19


In [15]:
dataset_name = 'fetaqa'
split = 'validation'
dataset = load_data_split(dataset_name,split)
tmp_save_path = 'datasets/schedule_test/tab_fact'
Check_Model_Data_Sequence = np.load(f'{tmp_save_path}/Check_Model_Data_Sequence_wo_check.npy',allow_pickle=True).item()
wikitq_df_processed = np.load(f'datasets/schedule_test/{dataset_name}/wikitq_df_processed.npy',allow_pickle=True).item()
wikitq_df = pd.DataFrame(dataset)

In [16]:
dataset_name = 'fetaqa'
split = 'validation'
dataset = load_data_split(dataset_name,split)

In [17]:
wikitq_df_processed = np.load('datasets/schedule_test/fetaqa/wikitq_df_processed.npy',allow_pickle=True).item()

In [3]:
from typing import List, Dict, Any
import pandas as pd
from transformers import AutoTokenizer

def find_long_samples(df_list: List[pd.DataFrame], 
                      tokenizer_path: str, 
                      threshold: int = 2000) -> List[Dict[str, Any]]:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    def serialize(df: pd.DataFrame) -> str:
        return " ".join(
            " | ".join([f"{col}: {val}" for col, val in row.items()])
            for _, row in df.iterrows()
        )

    results = []
    for idx in range(len(df_list)):
        df = df_list[idx]
        text = serialize(df)
        tokens = tokenizer(text, return_tensors=None)["input_ids"]
        token_len = len(tokens)
        if token_len > threshold:
            results.append(idx)
    
    return results

import pandas as pd
from typing import List, Tuple
from transformers import AutoTokenizer

def find_samples_by_length_from_dict(
    df_dict: Dict[Any, pd.DataFrame], 
    tokenizer_path: str
) -> Tuple[List[Any], List[Any], List[Any]]:
    """
    Categorizes DataFrame keys from a dictionary into three lists based on 
    the serialized token length of each DataFrame.

    Args:
        df_dict: A dictionary where keys are sample identifiers and values 
                 are pandas DataFrames.
        tokenizer_path: The path to the Hugging Face tokenizer.

    Returns:
        A tuple containing three lists of dictionary keys:
        - keys_short: Keys of samples with token length < 2000.
        - keys_medium: Keys of samples with token length between 2000 and 4000.
        - keys_long: Keys of samples with token length > 4000.
    """
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    def serialize(df: pd.DataFrame) -> str:
        """Helper function to convert a DataFrame to a single string."""
        return " ".join(
            " | ".join(f"{col}: {val}" for col, val in row.items())
            for _, row in df.iterrows()
        )

    # Initialize lists to store the keys for each category
    keys_short = []  # < 2000
    keys_medium = [] # 2000 to 4000
    keys_long = []   # > 4000

    # Iterate over the dictionary's items (key-value pairs)
    for key, df in df_dict.items():
        text = serialize(df)
        tokens = tokenizer.encode(text)
        token_len = len(tokens)

        # Categorize the key based on token length
        if token_len > 2000:
            keys_long.append(key)
        elif token_len >= 1000:
            keys_medium.append(key)
        else:
            keys_short.append(key)
    
    return keys_short, keys_medium, keys_long

# long_index = find_long_samples(wikitq_df_processed,tokenizer_path='../../../models/Qwen3-4B-Instruct-2507/',threshold=2000)
# samples_short, samples_medium, samples_long = find_samples_by_length_from_dict(wikitq_df_processed,tokenizer_path='../../../models/Qwen3-4B-Instruct-2507/')

In [24]:
len(samples_short), len(samples_medium), len(samples_long)

(1627, 300, 76)

In [56]:
## generate training-free router result
prompt_list = []
for index in range(len(dataset)):
    index = int(index)
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/training_free_router.txt',processed=True)
    prompt_list.append(prompt)
wikitq_df_rewrite_qa_semantic = wikitq_df
wikitq_df_rewrite_qa_semantic['instruction'] = prompt_list

In [57]:
wikitq_df_rewrite_qa_semantic.to_csv('../datasets/ablation/tab_fact_test_small_training_free_router.csv')

In [53]:
dataset_name = 'wikitq'
split = 'test'
dataset = load_data_split(dataset_name,split)
tmp_save_path = 'datasets/schedule_test/wikitq'
Check_Model_Data_Sequence = np.load(f'{tmp_save_path}/Check_Model_Data_Sequence_wo_check.npy',allow_pickle=True).item()
wikitq_df_processed = np.load(f'datasets/schedule_test/{dataset_name}/wikitq_df_processed.npy',allow_pickle=True).item()

In [ ]:
## generate training-free router result
prompt_list = []
for index in range():
    index = int(index)
    # rewritten_query = rewrite_query_list[index]
    # sequence = Check_Model_Data_Sequence[index]['Sequence']
    # insert_df = RAG_Hybrid[index]
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/training_free_router.txt',processed=True)
    # prompt = prompt.replace(dataset[index]['question'], rewritten_query.split('SPECIFIC')[0][:-1])
    # if sequence==[] or sequence.__contains__('Execute_SQL'):
    # evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
    # prompt = prompt + evidence
    prompt_list.append(prompt)
wikitq_df_rewrite_qa_semantic = wikitq_df.iloc[long_index]
wikitq_df_rewrite_qa_semantic['instruction'] = prompt_list

In [33]:
dataset_name = 'tab_fact'
split = 'test_small'
dataset = load_data_split(dataset_name,split)

In [67]:
ALL_LABELS = ['Base', 'Execute_SQL', 'RAG', 'Select_Column', 'Select_Row']
def parse_valid_route(input):
    try:
        parse = eval(input)[0]
        eval_parse = eval(parse)
        result = [e for e in eval_parse if e in ALL_LABELS]
    except:
        result = []
        for label in ALL_LABELS:
            if input.lower().__contains__(label.lower()):
                result.append(label)
    ## 如果有非法字符
    return result

In [68]:
training_free_router = pd.read_csv('../datasets/ablation/tab_fact_test_small_training_free_router_output_4B.csv',index_col=0)
training_free_router = [parse_valid_route(x) for x in training_free_router['predict'].tolist()]
count = []
for index in range(len(dataset)):
    valid_seq = [s for s in training_free_router[index] if s!='Base']
    count.append(len(valid_seq))
# count / len(dataset)
print(f'0 count number is {count.count(0)}, 1 count number is {count.count(1)}, 2 count number is {count.count(2)}, 3 count number is {count.count(3)}')

0 count number is 1301, 1 count number is 356, 2 count number is 297, 3 count number is 70


In [51]:
count = []
for index in range(len(dataset)):
    valid_seq = [s for s in Check_Model_Data_Sequence[index]['Sequence'] if s!='Base']
    count.append(len(valid_seq))
# count / len(dataset)

In [52]:
print(f'0 count number is {count.count(0)}, 1 count number is {count.count(1)}, 2 count number is {count.count(2)}, 3 count number is {count.count(3)}')

0 count number is 1447, 1 count number is 440, 2 count number is 51, 3 count number is 86


In [22]:
count_select = 0
count_all = 0
for index in range(len(dataset)):
    df = Check_Model_Data_Sequence[index]['data_entry']['table']
    count_select += df.shape[0] * df.shape[1]
    count_all += wikitq_df_processed[index].shape[0] * wikitq_df_processed[index].shape[1]
count_select / len(dataset)

81.50691699604744

127.70027624309392 wikitq
61.148480662983424 wikitq w/o Check
82.05138339920948 Tab_Fact
75.96 Tab_Fact w/o Check

seq_count_tab_fact
0 count number is 1463, 1 count number is 447, 2 count number is 52, 3 count number is 62
seql_count_tab_fact_wo_check
0 count number is 1447, 1 count number is 440, 2 count number is 51, 3 count number is 86
seq_count_wikitq
0 count number is 716, 1 count number is 2948, 2 count number is 143, 3 count number is 537
seq_count_wikitq_wo_check
0 count number is 927, 1 count number is 2693, 2 count number is 119, 3 count number is 605

In [1]:
import numpy as np

In [2]:
result = np.load('metrics_hyper_tau.npy',allow_pickle=True).item()

In [3]:
for key in result.keys():
    print(f"thres is {key}, Acc is {result[key]['accuracy']}")

thres is 0.8, Acc is 75.23020257826887
thres is 0.5, Acc is 75.1841620626151
thres is 0.2, Acc is 74.37845303867404
thres is 0.9, Acc is 58.057090239410684
thres is 0.99, Acc is 51.58839779005525


In [4]:
Check_Model_Data = np.load('datasets/schedule_test/wikitq/Check_Model_Data_Ablation.npy',allow_pickle=True).item()

In [10]:
seq_length = {}
for key in Check_Model_Data.keys():
    count = 0
    for index in Check_Model_Data[key].keys():
        seq = Check_Model_Data[key][index]['Sequence']
        seq_filter = [s for s in seq if s!='Base']
        count += len(seq_filter)
    seq_length[key] = count / len(Check_Model_Data[key].keys())
    print(f'for tau {key}, the average sequence length is {seq_length[key]}')

for tau 0.8, the average sequence length is 2.140193370165746
for tau 0.5, the average sequence length is 2.2285911602209945
for tau 0.2, the average sequence length is 2.2847605893186005
for tau 0.9, the average sequence length is 2.0757366482504604
for tau 0.99, the average sequence length is 1.8363259668508287


In [11]:
sample_num_metrics = {}
for sample_num in [1,2,3,4,5]:
    sample_num_metrics[sample_num] = {}
sample_num_metrics[1]['acc'] = 3319 / 4344
sample_num_metrics[2]['acc'] = 3362 / 4344
sample_num_metrics[3]['acc'] = 3367 / 4344
sample_num_metrics[4]['acc'] = 3376 / 4344
sample_num_metrics[5]['acc'] = 3376 / 4344
sample_num_metrics[1]['time'] = 9*60+41
sample_num_metrics[2]['time'] = 9*60+52
sample_num_metrics[3]['time'] = 10*60+7
sample_num_metrics[4]['time'] = 10*60+23
sample_num_metrics[5]['time'] = 10*60+22

In [16]:
sample_num_metrics

{1: {'acc': 0.7640423572744015,
  'time': 581,
  'total_time': 0.31652854511970535},
 2: {'acc': 0.7739410681399632,
  'time': 592,
  'total_time': 0.44129834254143646},
 3: {'acc': 0.7750920810313076, 'time': 607, 'total_time': 0.5669889502762431},
 4: {'acc': 0.7771639042357275, 'time': 623, 'total_time': 0.6929097605893186},
 5: {'acc': 0.7771639042357275, 'time': 622, 'total_time': 0.8149171270718232}}

In [15]:
sample_num_metrics[1]['total_time'] = (263 + 531 + 581) / 4344
sample_num_metrics[2]['total_time'] = (263 + 1062 + 592) / 4344
sample_num_metrics[3]['total_time'] = (263 + 1593 + 607) / 4344
sample_num_metrics[4]['total_time'] = (263 + 2124 + 623) / 4344
sample_num_metrics[5]['total_time'] = (263 + 2655 + 622) / 4344

In [17]:
np.save('sample_num_metrics.npy',sample_num_metrics)